In [1]:
import cv2
import pathlib
import numpy as np
import onnxruntime
from typing import Generator
import matplotlib.pyplot as plt
import re

MODEL_PATH: pathlib.Path = pathlib.Path("../data/logs/mlruns/669180362677009476/28c74b00b3c24059a887a64895e6dedf/artifacts/model/checkpoints/model_checkpoint/model_checkpoint.ckpt").resolve()
MODEL_PATH_ONNX: pathlib.Path = pathlib.Path("../data/logs/mlruns/669180362677009476/28c74b00b3c24059a887a64895e6dedf/artifacts/model/model.onnx").resolve()
TEST_SET_PATHS: Generator[pathlib.Path, None, None] = pathlib.Path("../data/test_set/").glob("*")

In [2]:
with pathlib.Path("../data/logs/mlruns/669180362677009476/28c74b00b3c24059a887a64895e6dedf/metrics/threshold").resolve().open("r") as f:
    THRESHOLD = round(float(f.readline().split(" ")[1]), 2)

In [3]:
ort_session: onnxruntime.InferenceSession = onnxruntime.InferenceSession(MODEL_PATH_ONNX)
input_name: str = ort_session.get_inputs()[0].name

In [5]:
OUTPUT_DIR = pathlib.Path("../data/test_set/predictions")

for video_path in TEST_SET_PATHS:
    frame_counter = 1
    vidcap = cv2.VideoCapture(str(video_path))
    
    while True:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (1000*frame_counter))    # added this line 
        success, image = vidcap.read()

        if not success:
            break
            
        print(image.shape)

        inputs = cv2.resize(image, (256, 256))
        inputs = inputs.transpose(2, 0, 1)
        inputs = np.expand_dims(inputs, axis=0)
        inputs = inputs.astype(np.float32)
        inputs /= 255.

        outputs = ort_session.run(None, {input_name: inputs})
        
        y_prob = 1/(1 + np.exp(-outputs[0].item()))
        y_class = y_prob >= THRESHOLD

        cv2.putText(
            img=image,
            text=f"Wild boar: {y_class} - Prediction: {y_prob:.3f}",
            org=(20, 50),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=1,
            color=(255, 255, 255),
            thickness=2,
        )
        frame_counter += 1
        
        image_name = re.sub(r"\W", "_", f"{video_path.name.lower()}_frame_{frame_counter}") + ".jpg"
        print(str(OUTPUT_DIR / image_name))

        cv2.imwrite(str(OUTPUT_DIR / image_name), image)

    vidcap.release()


(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_2.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_3.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_4.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_5.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_6.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_7.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_8.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_9.jpg
(1080, 1920, 3)
../data/test_set/predictions/large_male_wild_boar__tusker___zmcsyzwzi8o__webm_frame_10.jpg
(1080, 1920, 3)
../data/test_set/predictions/